In [2]:
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Hannanum
from konlpy.tag import Mecab
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import re 
# 형태소 분석을 위한 객체 생성.
kkma = Kkma()
komoran = Komoran()
hannanum = Hannanum()
mecab = Mecab()
okt = Okt()

SystemError: java.nio.file.InvalidPathException: Illegal char <*> at index 72: c:\users\momozzing\anaconda3\envs\nlp_env\lib\site-packages\konlpy\java\*

In [ ]:
text1 = "안녕 나는 윤호라고해"

In [ ]:
mecab.pos(text1)

In [ ]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [ ]:
train_data

In [ ]:
train_data.groupby("label").count()

In [ ]:
print(train_data.isnull().sum())
train_data = train_data.fillna(str(0))
print(train_data.isnull().sum())

In [ ]:
print(test_data.isnull().sum(0))
test_data = test_data.fillna(str(0))
print(test_data.isnull().sum(0))

In [ ]:
train_data = train_data.drop('id', axis=1)

In [ ]:
train_data

In [ ]:
test_data

In [ ]:
test_data = test_data.drop('id', axis=1)

In [ ]:
test_data

In [ ]:
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

stopwords=['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람', '주', '아니', '등', '같', '우리', '때', 
'년', '가', '한', '지', '대하', '오', '일', '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또',
'문제', '더', '사회', '많', '그리고', '크', '따르', '중', '나오', '가지', '씨', '시키', '만들', '지금', '생각하', '그러', '속', '하나', '집',
'살', '모르', '적', '월', '데', '자신', '안', '어떤', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이', '번', '나', 
'다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '정도', '좀', '원', '통하', '놓']

In [ ]:
train_data = train_data[:150000] 
X_train = []
for sentence in train_data['document']:
    temp_X = []
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_train.append(temp_X)

In [ ]:
test_data = test_data[:50000] 
X_test = []
for sentence in test_data['document']:
    temp_X = []
    temp_X = mecab.morphs(sentence) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
    X_test.append(temp_X)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
token = tokenizer.word_index

In [ ]:
token=mecab.morphs(token)  

In [ ]:
token=mecab.morphs(token)  

In [ ]:
token

In [ ]:
word2index={}  
bow=[]  
for voca in token:  
         if voca not in word2index.keys():  
             word2index[voca]=len(word2index)  
# token을 읽으면서, word2index에 없는 (not in) 단어는 새로 추가하고, 이미 있는 단어는 넘깁니다.   
             bow.insert(len(word2index)-1,1)
# BoW 전체에 전부 기본값 1을 넣어줍니다. 단어의 개수는 최소 1개 이상이기 때문입니다.  
         else:
            index=word2index.get(voca)
# 재등장하는 단어의 인덱스를 받아옵니다.
            bow[index]=bow[index]+1
# 재등장한 단어는 해당하는 인덱스의 위치에 1을 더해줍니다. (단어의 개수를 세는 것입니다.)  
print(word2index)  

In [ ]:
bow

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)


In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
Y_train = np.array(train_data['label'])
Y_test = np.array(test_data['label'])

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
X_train = np.delete(X_train, drop_train, axis=0)
Y_train = np.delete(Y_train, drop_train, axis=0)
print(len(X_train))
print(len(Y_train))

In [ ]:
print('리뷰의 최대 길이 :',max(len(l) for l in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(s) for s in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in nested_list:
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

In [ ]:

max_len = 30
below_threshold_len(max_len, X_train)

In [ ]:

X_train = pad_sequences(X_train, padding='post', maxlen = max_len)
X_test = pad_sequences(X_test, padding='post', maxlen = max_len)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
from sklearn.naive_bayes import GaussianNB
GauNB = GaussianNB()
GauNB.fit(X_train, Y_train)
GauNB_pred = GauNB.predict(X_test)

In [ ]:
print("정확도:", accuracy_score(Y_test, GauNB_pred))

In [ ]:
from sklearn.naive_bayes import BernoulliNB
BerNB = BernoulliNB()
BerNB.fit(X_train, Y_train)
BerNB_pred = BerNB.predict(X_test)

In [ ]:
print("정확도:", accuracy_score(Y_test, BerNB_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(max_depth=20, random_state=24)

rfc.fit(X_train, Y_train)

rfc_pred = rfc.predict(X_test)

In [ ]:
print("정확도:", accuracy_score(Y_test, rfc_pred))

In [ ]:
def sentiment_predict(new_sentence):
    new_sentence = mecab.morphs(new_sentence) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(rfc.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')

In [ ]:
sentiment_predict('와 개쩐다')

In [ ]:
sentiment_predict('이 영화 재밌다')

# 일단 이거는 binary classifier

이거를 풀기위해서 일단은 전처리 파트를 공부를 해야해요 

각자의 데이터셋을 만들어오기! <--- 중요!

               doucment
          3D/만 /아니었어도/ 별 /다섯 /개 줬을/텐데/.. 왜 /3D/로/ 나와서 /제 /심기/를 불편/하게/ 하/죠/?/?/  이런식으로 분류되게  만들어오기
          
          
전처리 끝내놓으면 이제 모델에 넣고 하기 편해가지고 전처리 파트 공부해서 전처리 하고 다시 모이기


In [ ]:
!pip install gensim